# Configurando o ambiente de Treino


## Libs necessárias

In [ ]:
'''!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2'''

In [ ]:

import os
import random
import numpy as np
import pandas as pd
from pprint import pprint
import tensorflow as tf
from kaggle_environments import make, evaluate
from gym import spaces

import matplotlib.pyplot as plt
from collections import deque
from tensorflow import keras
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from kaggle_environments.envs.hungry_geese.hungry_geese import translate, adjacent_positions, min_distance


## Classe para o ambiente do Ganso

In [ ]:
for action in Action:
    print(action)

In [ ]:
class HungryGeeseGym:
    def __init__(self, agent2="../input/coward-goose/submission-ralph-coward.py"):
        self.ks_env = make("hungry_geese", debug=False)
        self.env = self.ks_env.train([None, agent2,agent2,agent2])
        self.rows = self.ks_env.configuration.rows
        self.columns = self.ks_env.configuration.columns
        self.observation_space_size = self.rows * self.columns
        # Learn about spaces here: http://gym.openai.com/docs/#spaces
        # Permitted actions
        # ['NORTH', 'EAST', 'SOUTH', 'WEST']
        actL = []
        self.debug = True
        for action in Action:
            actL.append(action)
        self.actions = actL
        
        #print(actL)
        # Defined Action Space(Must)
        self.action_space = spaces.Discrete(len(self.actions))
        
        self.observation_space = spaces.Box(low=np.zeros(shape=(11,), dtype=int), high=np.zeros(shape=(11,), dtype=int)+10)
        
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1000)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None
        self.last = self.actions[self.action_space.sample()]
    def reset(self):
        self.last = self.actions[self.action_space.sample()]
        
        self.obs = self.env.reset()
        #print(self.obs)
        return self.sensors(self.get_grid(self.obs))
    
    def opposite(self,action):
        if action == Action.NORTH:
            return Action.SOUTH
        if action == Action.SOUTH:
            return Action.NORTH
        if action == Action.EAST:
            return Action.WEST
        if action == Action.WEST:
            return Action.EAST
        
    def step(self, action):
        action = self.actions[action]
        # Check if agent's move is valid
        is_valid = action != self.last


        
        self.last = self.opposite(action)


        cont = len(self.obs['geese'][self.obs['index']])
        #print(cont)
        reward, done, _ = -50, True, {}
        if is_valid: # Play the move
            self.obs, old_reward, done, _ = self.env.step(action.name)
            #print(self.obs)
            if(done):
                reward = 100 # se ganhou
                if(self.obs['geese'][self.obs['index']] == []):
                    reward = -5 # se perdeu
            else:
                if(len(self.obs['geese'][self.obs['index']])>cont ):
                    reward = 20 # se comeu
                else:
                    if(len(self.obs['geese'][self.obs['index']])<cont ):
                        reward = -10 
                        #print("Step: "+str(self.obs['step']) + " /  Perdeu tamanho... " + str(reward)+" Pontos")
                    else:
                        reward = -1 # se não fez nada


                
        grid = self.get_grid(self.obs)
        
        sensores = self.sensors(grid)
        if(self.debug):
            print(sensores.shape)
            print(sensores)
            print(grid.reshape(1,-1)[0].shape)

        # o obs tem que ser [indice da ultima ação. [sens1], [sens2], [sens3]]
        return sensores, reward, done, _
    
    def sensors(self,grid):
       
        if(len(self.obs['geese'][self.obs['index']])==0):
            return np.array([0,0,0,0,0,0,0,0,0,0,0])
        px,py = row_col(self.obs['geese'][self.obs['index']][0],self.columns)
        actL = []
        for action in Action:
            actL.append(action)
        frente = 0
        if(self.last):
            for i in range(0,len(actL)):
                if(actL[i] == self.opposite(self.last)):
                    frente = i
                    break
            actL.remove(self.last)
        else: # Diz que esta indo para o norte e remove o oposto (SUL)
            actL.remove(action.SOUTH)
            frente = 0
       

        direita = frente+1
        if(direita>3):
            direita = direita-4
        esquerda = frente + 3
        if(esquerda>3):
            esquerda = esquerda - 4

        movimentos = [
            [-1,0],     #Norte
            [0,1],      #Leste
            [1,0],      #Sul
            [0,-1],     #Oeste
        ]
        sensor_frente = [0,0] #[distancia,tipo (0:inimigo,1:food)]
        for i in range(0,11):
            px_a,py_a = (px+movimentos[frente][0]*(i+1),py+movimentos[frente][1]*(i+1))
            if(px_a>6):
                px_a = px_a - 7
            if(py_a>10):
                py_a = py_a - 11
            
            if(px_a<0):
                px_a = 7 + px_a
            if(py_a<0):
                py_a = 11 + py_a
            
            if(grid[px_a][py_a] == 2 or grid[px_a][py_a] == 1):
                break
            if(grid[px_a][py_a] == 3):
                sensor_frente[1] = 1
                break
            else:
                sensor_frente[0]+=1
        
        sensor_esquerda = [0,0] #[distancia,tipo (0:inimigo,1:food)]
        for i in range(0,11):
            px_a,py_a = (px+movimentos[esquerda][0]*(i+1),py+movimentos[esquerda][1]*(i+1))
            if(px_a>6):
                px_a = px_a - 7
            if(py_a>10):
                py_a = py_a - 11
            
            if(px_a<0):
                px_a = 7+ px_a
            if(py_a<0):
                py_a = 11 + py_a
            
            if(grid[px_a][py_a] == 2 or grid[px_a][py_a] == 1):
                break
            if(grid[px_a][py_a] == 3):
                sensor_esquerda[1] = 1
                break
            else:
                sensor_esquerda[0]+=1
        
        sensor_direita = [0,0] #[distancia,tipo (0:inimigo,1:food)]
        for i in range(0,11):
            px_a,py_a = (px+movimentos[direita][0]*(i+1),py+movimentos[direita][1]*(i+1))
            if(px_a>6):
                px_a = px_a - 7
            if(py_a>10):
                py_a = py_a - 11
            
            if(px_a<0):
                px_a = 7 + px_a
            if(py_a<0):
                py_a = 11 + py_a
            
            
            if(grid[px_a][py_a] == 2 or grid[px_a][py_a] == 1):
                break
            if(grid[px_a][py_a] == 3):
                sensor_direita[1] = 1
                break
            else:
                sensor_direita[0]+=1
        
        # Verificando as diagonais
        tras = frente + 2
        if(tras>=4):
            tras-=4
        
        px_a,py_a = (px+    movimentos[direita][0]  + movimentos[frente][0] )  ,  (py+     movimentos[direita][1]  + movimentos[frente][1])   
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7+ px_a
        if(py_a<0):
            py_a = 11 + py_a
        frente_direita = grid[px_a,py_a]

        px_a,py_a =   (px+    movimentos[esquerda][0] + movimentos[frente][0] ) ,   (py+     movimentos[esquerda][1] + movimentos[frente][1])
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7+ px_a
        if(py_a<0):
            py_a = 11 + py_a
        frente_esquerda = grid[px_a,py_a]

        px_a,py_a =      (px+    movimentos[esquerda][0] + movimentos[tras][0]   ) ,   (py+     movimentos[esquerda][1] + movimentos[tras][1]  )
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7+ px_a
        if(py_a<0):
            py_a = 11 + py_a
        tras_esqueda = grid[px_a,py_a]


        px_a,py_a=      (px+    movimentos[direita][0]  + movimentos[tras][0]   ) ,   (py+     movimentos[direita][1]  + movimentos[tras][1]  )
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7+ px_a
        if(py_a<0):
            py_a = 11 + py_a
        tras_direita = grid[px_a,py_a]

        if(self.debug):
            print("x:"+str(px)+" / y:"+str(py))
            print("Frente ("+str(self.actions[frente].name) +"):"+str(sensor_frente) + " / Esquerda ("+str(self.actions[esquerda].name) +"):"+str(sensor_esquerda) + " / Direita ("+str(self.actions[direita].name) +"):"+str(sensor_direita))
            print(grid)
        return np.array([frente,sensor_frente[0],sensor_frente[1],sensor_esquerda[0],sensor_esquerda[1],sensor_direita[0],sensor_direita[1],frente_direita,frente_esquerda,tras_direita,tras_esqueda])

    def get_grid(self,obs):
        mapa = []
        for i in range(0,self.rows):
            mapa.append([])
            for j in range(0,self.columns):
                achou = False
                for food in obs['food']:
                    x,y = row_col(food,self.columns)
                    if(x == i and y == j):
                        mapa[i].append(3)
                        achou = True
                        break
                if(achou):
                    continue
                aux = 0
                for goose in obs['geese']:
                    gs = 2
                    if(aux == obs['index']):
                        gs = 1
                    aux = aux +1
                    for part in goose:
                        x,y = row_col(part,self.columns)
                        if(x == i and y == j):
                            mapa[i].append(gs)
                            achou = True
                            break
                    if(achou):
                        break
                if(achou):
                    continue
                mapa[i].append(0)

        return np.array(mapa)

# Criando a RNA e Treinando

In [ ]:
class DeepQNetwork():
    def __init__(self, states, actions, alpha, gamma, epsilon,epsilon_min, epsilon_decay,layers_num,layer_neuron_num):
        self.nS = states
        self.nA = actions
        self.memory = deque([], maxlen=2500)
        self.alpha = alpha
        self.gamma = gamma
        #Explore/Exploit
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.model = self.build_model(layers_num,layer_neuron_num)
        self.loss = []
        
    def build_model(self,layers_num,layer_neuron_num):
        model = keras.Sequential() #linear stack of layers https://keras.io/models/sequential/
        model.add(keras.layers.Dense(11, input_dim=self.nS, activation='relu')) #[Input]
        for i in range(0,layers_num):
            model.add(keras.layers.Dense(layer_neuron_num, activation='relu'))
        model.add(keras.layers.Dense(self.nA, activation='linear')) #[output]
        #   Size has to match the output (different actions)
        #   Linear activation on the last layer
        model.compile(loss='mean_squared_error', #Loss function: Mean Squared Error
                      optimizer=keras.optimizers.Adam(lr=self.alpha)) #Optimaizer: Adam (Feel free to check other options)
        return model

    def action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.nA) #Explore
        action_vals = self.model.predict(state) #Exploit: Use the NN to predict the correct action from this state
        return np.argmax(action_vals[0])

    def test_action(self, state): #Exploit
        action_vals = self.model.predict(state)
        return np.argmax(action_vals[0])

    def store(self, state, action, reward, nstate, done):
        #Store the experience in memory
        self.memory.append( (state, action, reward, nstate, done) )

    def experience_replay(self, batch_size):
        #Execute the experience replay
        minibatch = random.sample( self.memory, batch_size ) #Randomly sample from memory

        #Convert to numpy for speed by vectorization
        x = []
        y = []
        np_array = np.array(minibatch,dtype=object)
        st = np.zeros((0,self.nS)) #States
        nst = np.zeros( (0,self.nS) )#Next States
        for i in range(len(np_array)): #Creating the state and next state np arrays
            st = np.append( st, np_array[i,0], axis=0)
            nst = np.append( nst, np_array[i,3], axis=0)
        st_predict = self.model.predict(st) #Here is the speedup! I can predict on the ENTIRE batch
        nst_predict = self.model.predict(nst)
        index = 0
        for state, action, reward, nstate, done in minibatch:
            x.append(state)
            #Predict from state
            nst_action_predict_model = nst_predict[index]
            if done == True: #Terminal: Just assign reward much like {* (not done) - QB[state][action]}
                target = reward
            else:   #Non terminal
                target = reward + self.gamma * np.amax(nst_action_predict_model)
            target_f = st_predict[index]
            target_f[action] = target
            y.append(target_f)
            index += 1
        #Reshape for Keras Fit
        x_reshape = np.array(x).reshape(batch_size,self.nS)
        y_reshape = np.array(y)
        epoch_count = 1 #Epochs is the number or iterations
        hist = self.model.fit(x_reshape, y_reshape, epochs=epoch_count, verbose=0)
        #Graph Losses
        for i in range(epoch_count):
            self.loss.append( hist.history['loss'][i] )
        #Decay Epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

## Treino com DQN

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
import math
def train(layers_num,layer_neuron_num):

    try:
        os.mkdir("./data/"+str(layers_num)+'-'+str(layer_neuron_num))
        print("-------- Iniciando treino para o modelo com "+str(layers_num)+" Hidden Layers e com "+str(layer_neuron_num)+" Neuronios em cada Layer --------")
    except:
        print("Treino ja Executado anteriormente... Tudo será sobrescrito!")
        
    env = HungryGeeseGym()

    #Global Variables
    EPISODES = 5000
    TRAIN_END = 0
    #Hyper Parameters
    discount_rate = 0.95 #Gamma
    learning_rate = 0.0001 #Alpha
    batch_size = 24 #Size of the batch used in the experience replay

    #Create the agent
    nS = env.observation_space.shape[0]
    nA = env.action_space.n
    try:
        del dqn
    except:
        discount_rate
    dqn = DeepQNetwork(nS, nA, learning_rate, discount_rate, 1, 0.001, (0.222)*(1/(EPISODES/2)),layers_num,layer_neuron_num )
    #Training
    rewards = [] #Store rewards for graphing
    epsilons = [] # Store the Explore/Exploit
    TEST_Episodes = 0
    
    # Para deixar sem printar nada no ambiente
    env.debug = False
    
    for e in tqdm(range(EPISODES),desc = (str(layers_num)+'-'+str(layer_neuron_num)),leave = False):
        #if(e%int(EPISODES/100) == 0):
        #    print(str(layers_num)+'-'+str(layer_neuron_num)+": "+str(math.floor(100*e/EPISODES)) + "%")
        state = env.reset()
        state = np.reshape(state, [1, nS]) # Resize to store in memory to pass to .predict
        tot_rewards = 0
        for time in range(200): 
            action = dqn.action(state)
            nstate, reward, done, _ = env.step(action)
            nstate = np.reshape(nstate, [1, nS])
            tot_rewards += reward
            dqn.store(state, action, reward, nstate, done) # Resize to store in memory to pass to .predict
            state = nstate
            if done:
                rewards.append(tot_rewards)
                epsilons.append(dqn.epsilon)
                '''print("episode: {}/{}, score: {}, e: {}"
                    .format(e, EPISODES, tot_rewards, dqn.epsilon))'''
                break
            #Experience Replay
            if len(dqn.memory) > batch_size:
                dqn.experience_replay(batch_size)
    
    dqn.model.save('./data/'+str(layers_num)+'-'+str(layer_neuron_num)+'/model')
    df0 = pd.DataFrame()
    df0['Reward'] = np.array(rewards)
    df0.to_csv('./data/'+str(layers_num)+'-'+str(layer_neuron_num)+"/modelData-total.csv")
    med = 0
    y_values = []
    x_values = []
    eps_graph = []
    eps = 0
    aux = 0
    BATCH = 10
    for i in range(0,len(rewards)): 
        med+=rewards[i]
        eps+=epsilons[i]
        if i%(BATCH-1) == 0 and i != 0:
            x_values.append(aux)
            aux+=1
            y_values.append(med/BATCH)
            eps_graph.append(40*eps/BATCH)
            eps = 0
            med = 0

    df = pd.DataFrame()
    #print(x_values)
    df["Score"] = np.array(y_values)
    df['Epslon'] = np.array(eps_graph)
    df["round"] = np.array(x_values)
    fig = px.line(df, x='round', y=['Score','Epslon'])
    fig.show()
    df.to_csv('./data/'+str(layers_num)+'-'+str(layer_neuron_num)+"/modelData.csv")
    fig.write_html('./data/'+str(layers_num)+'-'+str(layer_neuron_num)+"plot.html")



In [ ]:
train(7,49)

# Teste

In [ ]:
actions = []
for act in Action:
        actions.append(act)
def get_grid_from_obs(obs,columns,rows):
        mapa = []
        for i in range(0,rows):
            mapa.append([])
            for j in range(0,columns):
                achou = False
                for food in obs['food']:
                    x,y = row_col(food,columns)
                    if(x == i and y == j):
                        mapa[i].append(3)
                        achou = True
                        break
                if(achou):
                    continue
                aux = 0
                for goose in obs['geese']:
                    gs = 2
                    if(aux == obs['index']):
                        gs = 1
                    aux = aux +1
                    for part in goose:
                        x,y = row_col(part,columns)
                        if(x == i and y == j):
                            mapa[i].append(gs)
                            achou = True
                            break
                    if(achou):
                        break
                if(achou):
                    continue
                mapa[i].append(0)
        return np.array(mapa)
def opposite(action):
    if action == Action.NORTH:
        return Action.SOUTH
    if action == Action.SOUTH:
        return Action.NORTH
    if action == Action.EAST:
        return Action.WEST
    if action == Action.WEST:
        return Action.EAST
def get_sensors_from_grid(grid,columns,obs,last,debug):
    print()
    
    global actions
    if(len(obs['geese'][obs['index']])==0):
        return np.array([0,0,0,0,0,0,0])
    px,py = row_col(obs['geese'][obs['index']][0],columns)
    actL = []
    for action in Action:
        actL.append(action)
    frente = 0
    if(debug):
        print(last.name)

    if(last):
        for i in range(0,len(actL)):
            if(actL[i] == opposite(last)):
                frente = i
                break
        actL.remove(last)
    else: # Diz que esta indo para o norte e remove o oposto (SUL)
        actL.remove(action.SOUTH)
        frente = 0
    

    direita = frente+1
    if(direita>3):
        direita = direita-4
    esquerda = frente + 3
    if(esquerda>3):
        esquerda = esquerda - 4

    movimentos = [
        [-1,0],     #Norte
        [0,1],     #Leste
        [1,0],      #Sul
        [0,-1],      #Oeste
    ]
    sensor_frente = [0,0] #[distancia,tipo (0:inimigo,1:food)]
    for i in range(0,11):
        px_a,py_a = (px+movimentos[frente][0]*(i+1),py+movimentos[frente][1]*(i+1))
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7 + px_a
        if(py_a<0):
            py_a = 11 + py_a
        
        if(grid[px_a][py_a] == 2 or grid[px_a][py_a] == 1):
            break
        if(grid[px_a][py_a] == 3):
            sensor_frente[1] = 1
            break
        else:
            sensor_frente[0]+=1
    
    sensor_esquerda = [0,0] #[distancia,tipo (0:inimigo,1:food)]
    for i in range(0,11):
        px_a,py_a = (px+movimentos[esquerda][0]*(i+1),py+movimentos[esquerda][1]*(i+1))
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7+ px_a
        if(py_a<0):
            py_a = 11 + py_a
        
        if(grid[px_a][py_a] == 2 or grid[px_a][py_a] == 1):
            break
        if(grid[px_a][py_a] == 3):
            sensor_esquerda[1] = 1
            break
        else:
            sensor_esquerda[0]+=1
    
    sensor_direita = [0,0] #[distancia,tipo (0:inimigo,1:food)]
    for i in range(0,11):
        px_a,py_a = (px+movimentos[direita][0]*(i+1),py+movimentos[direita][1]*(i+1))
        if(px_a>6):
            px_a = px_a - 7
        if(py_a>10):
            py_a = py_a - 11
        
        if(px_a<0):
            px_a = 7 + px_a
        if(py_a<0):
            py_a = 11 + py_a
        
        
        if(grid[px_a][py_a] == 2 or grid[px_a][py_a] == 1):
            break
        if(grid[px_a][py_a] == 3):
            sensor_direita[1] = 1
            break
        else:
            sensor_direita[0]+=1
    
    # Verificando as diagonais
    tras = frente + 2
    if(tras>=4):
        tras-=4
    
    px_a,py_a = (px+    movimentos[direita][0]  + movimentos[frente][0] )  ,  (py+     movimentos[direita][1]  + movimentos[frente][1])   
    if(px_a>6):
        px_a = px_a - 7
    if(py_a>10):
        py_a = py_a - 11
    
    if(px_a<0):
        px_a = 7+ px_a
    if(py_a<0):
        py_a = 11 + py_a
    frente_direita = grid[px_a,py_a]

    px_a,py_a =   (px+    movimentos[esquerda][0] + movimentos[frente][0] ) ,   (py+     movimentos[esquerda][1] + movimentos[frente][1])
    if(px_a>6):
        px_a = px_a - 7
    if(py_a>10):
        py_a = py_a - 11
    
    if(px_a<0):
        px_a = 7+ px_a
    if(py_a<0):
        py_a = 11 + py_a
    frente_esquerda = grid[px_a,py_a]

    px_a,py_a =      (px+    movimentos[esquerda][0] + movimentos[tras][0]   ) ,   (py+     movimentos[esquerda][1] + movimentos[tras][1]  )
    if(px_a>6):
        px_a = px_a - 7
    if(py_a>10):
        py_a = py_a - 11
    
    if(px_a<0):
        px_a = 7+ px_a
    if(py_a<0):
        py_a = 11 + py_a
    tras_esqueda = grid[px_a,py_a]


    px_a,py_a=      (px+    movimentos[direita][0]  + movimentos[tras][0]   ) ,   (py+     movimentos[direita][1]  + movimentos[tras][1]  )
    if(px_a>6):
        px_a = px_a - 7
    if(py_a>10):
        py_a = py_a - 11
    
    if(px_a<0):
        px_a = 7+ px_a
    if(py_a<0):
        py_a = 11 + py_a
    tras_direita = grid[px_a,py_a]
    if(debug):
        print("x:"+str(px)+" / y:"+str(py))
        print("Frente ("+str(actions[frente].name) +"):"+str(sensor_frente) + " / Esquerda ("+str(actions[esquerda].name) +"):"+str(sensor_esquerda) + " / Direita ("+str(actions[direita].name) +"):"+str(sensor_direita))
        print(grid)
    return np.array([frente,sensor_frente[0],sensor_frente[1],sensor_esquerda[0],sensor_esquerda[1],sensor_direita[0],sensor_direita[1],frente_direita,frente_esquerda,tras_direita,tras_esqueda])
model = keras.models.load_model('./data/7-49/model')
last = Action.SOUTH
def agent(obs,config):
    global last
    global model
    state = get_grid_from_obs(obs,config.columns,config.rows)
    state = get_sensors_from_grid(state,config.columns,obs,last,True)
    print()
    state = np.reshape(state, [1, 11])
    print(state)
    action =np.argmax(model.predict(state)[0]) 
    actL = []
    for act in Action:
            actL.append(act)
    print(actL)
    last = opposite(actL[action])
    return actL[action].name
    

In [ ]:
env1 = make("hungry_geese", debug=False) #set debug to True to see agent internals each step

env1.reset()
op1 = [agent,"../input/coward-goose/submission-ralph-coward.py", "../input/coward-goose/submission-ralph-coward.py","../input/coward-goose/submission-ralph-coward.py"]
op2 = [agent,agent,agent,agent]
env1.run(op1)
env1.render(mode="ipython",width=700, height=600)